# Chapter 20. Attribute Descriptors
> Learning about descriptors not only provides access to a larger toolset, it creates a deeper understanding of how Python workds and an appreciation for the elegance of its design.   
<br>Raymond Hettinger, *Python core developer and guru*

<br>

Protocol
- `__get__`
- `__set__`
- `__delete__`

# Descriptor Example: Attribute Validation

## LineItem Take #3: A Simple Descriptor

In [3]:
class Quantity:
    def __init__(self, storage_name):
        self.storage_name = storage_name
    
    def __set__(self, instance, value):
        if value > 0:
            # Need to ccess storage_name of instance by __dict__
            # to avoid entering infinite loop.
            # e.g, instance.storage_name, setattr(instance, self.storage_name, value)
            # calls __set__ again and again.
            instance.__dict__[self.storage_name] = value
        else:
            raise ValueError('value must be > 0')

class LineItem:
    weight = Quantity('weight')
    price = Quantity('price')
    
    def __init__(self, description, weight, price):
        self.description = description
        self.weight = weight
        self.price = price
    
    def subtotal(self):
        return self.weight * self.price

## LineItem Take #4: Automatic Storage Attribute Names

In [8]:
class Quantity:
    __counter = 0
    
    def __init__(self):
        cls = self.__class__
        prefix = cls.__name__
        index = cls.__counter
        self.storage_name = '_{}#{}'.format(prefix, index)
        cls.__counter += 1
    
    def __get__(self, instance, owner):
        return getattr(instance, self.storage_name)
    
    def __set__(self, instance, value):
        if value > 0:
            # No need to care of infinite loop
            # because storage_name is different between descriptor and LineItem instance
            setattr(instance, self.storage_name, value)
        else:
            raise ValueError('value must be > 0')

class Lineitem:
    weight = Quantity()
    price = Quantity()
    
    def __init__(self, description, weight, price):
        self.description = description
        self.weight = weight
        self.price = price
    
    def subtotal(self):
        return self.weight * self.price

In [9]:
coconuts = LineItem('Brazilian coconut', 20, 17.95)
coconuts.weight, coconuts.price

(20, 17.95)

In [10]:
getattr(coconuts, '_Quantity#0'), getattr(coconuts, '_Quantity#1')

AttributeError: 'LineItem' object has no attribute '_Quantity#0'

In [11]:
class Quantity:
    __counter = 0

    def __init__(self):
        cls = self.__class__
        prefix = cls.__name__
        index = cls.__counter
        self.storage_name = '_{}#{}'.format(prefix, index)
        cls.__counter += 1

    def __get__(self, instance, owner):
        # return self to represent that you access to descriptor
        if instance is None:
            return self
        else:
            return getattr(instance, self.storage_name)

    def __set__(self, instance, value):
        if value > 0:
            setattr(instance, self.storage_name)
        else:
            raise ValueError('value must be > 0')

In [12]:
LineItem.price

In [13]:
br_nuts = LineItem('Brazil nuts', 10, 34.95)
br_nuts.price

34.95

In [ ]:
# Descriptor as utitlity
class LineItem:
    weight = model.Quantity()
    price = model.Quantity()
    
    def __init__(self, description, weight, price):
        self.description = description
        self.weight = weight
        self.price = price
    
    def subtotal(self):
        return self.weight * self.price

## LineItem Take #5: A New Descriptor Type

In [14]:
import abc


class AutoStorage:
    __counter = 0
    
    def __init__(self):
        cls = self.__class__
        prefix = cls.__name__
        index = cls.__counter
        self.storage_name = '_{}#{}'.format(prefix, index)
        cls.__counter += 1
    
    def __get__(self, instance, owner):
        if instance is None:
            return self
        else:
            return getattr(instance, self.storage_name)
        
    def __set__(self, instance, value):
        setattr(instance, self.storage_name, value)


class Validated(abc.ABC, AutoStorage):
    def __set__(self, instance, value):
        value = self.validate(instance, value)
        super().__set__(instance, value)
    
    @abc.abstractmethod
    def validate(self, instance, value):
        """return validated value or raise Value Error"""
    

class Quantity(Validated):
    """a number greater than zero"""
    
    def validate(self, instance, value):
        if value <= 0:
            raise ValueError('value must be > 0')
        return value


class NonBlank(Validated):
    """a string with at least one non-space character"""
    
    def validate(self, instance, value):
        value = value.strip()
        if len(value) == 0:
            raise ValueError('value cannot be empty or blank')
        return value

# Overriding Versus Nonoverriding Descriptors
`overriding descriptor`
- implements the `__set__` method
- e.g, `properties`

<br>

a.k.a.  
- overriding descriptors
    - data descriptors or enforced descriptors  
- nonoverriding descriptors
    - nondata descriptors or shadowable descriptors

In [33]:
def cls_name(obj_or_cls):
    cls = type(obj_or_cls)
    if cls is type:
        cls = obj_or_cls
    return cls.__name__.split('.')[-1]

def display(obj):
    cls = type(obj)
    if cls is type:
        return '<class {}>'.format(obj.__name__)
    elif cls in [type(None), int]:
        return repr(obj)
    else:
        return '<{} object>'.format(cls_name(obj))

def print_args(name, *args):
    pseudo_args = ', '.join(display(x) for x in args)
    print('-> {}.__{}__({})'.format(cls_name(args[0]), name, pseudo_args))


class Overriding:
    """a.k.a. data descriptor or enforced descriptor"""
    
    def __get__(self, instance, owner):
        print_args('get', self, instance, owner)
    
    def __set__(self, instance, value):
        print_args('set', self, instance, value)
    

class OverridingNoGet:
    """an overriding descriptor without ``__get__``"""
    
    def __set__(self, instance, value):
        print_args('set', self, instance, value)


class NonOverriding:
    """a.k.a. non-data or shadowable descriptor"""
    
    def __get__(self, instance, owner):
        print_args('get', self, instance, owner)

        
class Managed:
    over = Overriding()
    over_no_get = OverridingNoGet()
    non_over = NonOverriding()
    
    def spam(self):
        print('-> Managed.spam({})'.format(display(self)))

In [17]:
class MyClass:
    pass

c = MyClass()
type(c)

__main__.MyClass

In [18]:
c

In [19]:
type(MyClass)

type

In [34]:
obj = Managed()
obj.over

-> Overriding.__get__(<Overriding object>, <Managed object>, <class Managed>)


In [21]:
Managed.over

-> Overriding.__get__(<Overriding object>, None, <class Managed>)


In [22]:
obj.over = 7

-> Overriding.__set__(<Overriding object>, <Managed object>, 7)


In [23]:
obj.over

-> Overriding.__get__(<Overriding object>, <Managed object>, <class Managed>)


In [24]:
obj.__dict__['over'] = 8
vars(obj)

{'over': 8}

In [25]:
obj.over

-> Overriding.__get__(<Overriding object>, <Managed object>, <class Managed>)


## Overriding Descriptor Without __get__

In [35]:
obj.over_no_get

In [36]:
Managed.over_no_get

In [37]:
obj.over_no_get = 7

-> OverridingNoGet.__set__(<OverridingNoGet object>, <Managed object>, 7)


In [38]:
obj.over_no_get

In [40]:
obj.__dict__['over_no_get'] = 9
obj.over_no_get

9

In [41]:
obj.over_no_get = 7

-> OverridingNoGet.__set__(<OverridingNoGet object>, <Managed object>, 7)


In [42]:
obj.over_no_get

9

## Overriding a Descriptor in the Class

In [43]:
obj = Managed()
Managed.over = 1
Managed.over_no_get = 2
Managed.non_over = 3
obj.over, obj.over_no_get, obj.non_over

(1, 2, 3)

# Methods Are Descriptors
- Functions within a class becomes a bound method because all user-defined functions have a `__get__` method
    - but do not implement `__set__`, so those functions are **nonoverriding descriptors**
- bound method object
    - implements attributes
        - `__get__`
        - `__self__`
            - holds reference to the instance
        - `__func__`
            - holds reference to the original function
        - `__call__`
            - invokes the original function (`__func__`) with the bound instance (`__self__`)
       

In [44]:
obj = Managed()
obj.spam

<bound method Managed.spam of <__main__.Managed object at 0x109090390>>

In [45]:
Managed.spam

<function __main__.Managed.spam(self)>

In [46]:
obj.spam = 7
obj.spam

7

In [47]:
import collections


class Text(collections.UserString):
    def __repr__(self):
        return 'Text({!r})'.format(self.data)
    
    def reverse(self):
        return self[::-1]

In [48]:
word = Text('forward')
word

Text('forward')

In [49]:
word.reverse()

Text('drawrof')

In [50]:
type(Text.reverse), type(word.reverse)

(function, method)

In [51]:
list(map(Text.reverse, ['repaid', (10, 20, 30), Text('stressed')]))

['diaper', (30, 20, 10), Text('desserts')]

In [52]:
Text.reverse.__get__(word)

<bound method Text.reverse of Text('forward')>

In [53]:
Text.reverse.__get__(None, Text)

<function __main__.Text.reverse(self)>

In [54]:
word.reverse

<bound method Text.reverse of Text('forward')>

In [55]:
word.reverse.__self__

Text('forward')

In [56]:
word.reverse.__func__ is Text.reverse

True

# Descriptor Usage Tips
- Use `property` to Keep It Simple
- Read-only descriptors require `__set__`
- Validation descriptors can work with `__set__` only
- Caching can be done efficiently with `__get__` only
- Nonspecial methods can be shadowed by instannce attributes